## Theoretical Overview

$$ H(x) = Wx + b $$

$$ cost(W, b) = \frac{1}{m} \sum^m_{i=1} \left( H(x^{(i)}) - y^{(i)} \right)^2 $$

 - $H(x)$: 주어진 $x$ 값에 대해 예측을 어떻게 할 것인가
 - $cost(W, b)$: $H(x)$ 가 $y$ 를 얼마나 잘 예측했는가

## Import Inquired Libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility - random seed를 통제해주기 위해서
torch.manual_seed(1)

## Data 

In [3]:
x_train = torch.FloatTensor([[1],[2],[3]])   # Input data
y_train = torch.FloatTensor([[1],[2],[3]])   # Output data

In [4]:
print(x_train)
print(x_train.shape)

tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])


In [5]:
print(y_train)
print(y_train.shape)

tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])


기본적으로 Pytorch는 NCHW([batch, channels, height, width]) 형태이다.

## Weight Initialization

In [7]:
W = torch.zeros(1, requires_grad=True)   # 0으로 초기화, 학습을 시켜줄 것이라 명시
print(W)

tensor([0.], requires_grad=True)


In [8]:
b = torch.zeros(1, requires_grad=True)   # 0으로 초기화, 학습을 시켜줄 것이라 명시
print(b)

tensor([0.], requires_grad=True)


## Hypothesis

우리의 모델 - 하나의 직선을 찾는 일  
가장 최적의 직선을 표현하는 W, b를 찾는 것이 우리의 목표!

$$ H(x) = Wx + b $$

In [10]:
hypothesis = x_train * W + b
print(hypothesis)

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)


## Cost

$$ cost(W, b) = \frac{1}{m} \sum^m_{i=1} \left( H(x^{(i)}) - y^{(i)} \right)^2 $$

In [11]:
print(hypothesis - y_train)

tensor([[-1.],
        [-2.],
        [-3.]], grad_fn=<SubBackward0>)


In [12]:
cost = torch.mean((hypothesis - y_train)**2)
print(cost)

tensor(4.6667, grad_fn=<MeanBackward0>)


## Gradient Descent

In [13]:
optimizer = optim.SGD([W,b], lr= 0.01)

**항상 붙어다니는 3줄!!**

In [14]:
optimizer.zero_grad()      # 계산했던 기울기의 값이 남아 누적되지 않도록 초기화
cost.backward()            # backpropagation으로 기울기 계산
optimizer.step()           # step()으로 개선

In [15]:
print(W)
print(b)

tensor([0.0933], requires_grad=True)
tensor([0.0400], requires_grad=True)


Let's check if the hypothesis is now better.

In [17]:
hypothesis = x_train * W + b
print(hypothesis)
print()
cost = torch.mean((hypothesis - y_train) ** 2)
print(cost)

tensor([[0.1333],
        [0.2267],
        [0.3200]], grad_fn=<AddBackward0>)

tensor(3.6927, grad_fn=<MeanBackward0>)


## Training with Full Code

In [18]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])
# 모델 초기화
W = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train * W + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))

Epoch    0/1000 W: 0.093, b: 0.040 Cost: 4.666667
Epoch  100/1000 W: 0.873, b: 0.289 Cost: 0.012043
Epoch  200/1000 W: 0.900, b: 0.227 Cost: 0.007442
Epoch  300/1000 W: 0.921, b: 0.179 Cost: 0.004598
Epoch  400/1000 W: 0.938, b: 0.140 Cost: 0.002842
Epoch  500/1000 W: 0.951, b: 0.110 Cost: 0.001756
Epoch  600/1000 W: 0.962, b: 0.087 Cost: 0.001085
Epoch  700/1000 W: 0.970, b: 0.068 Cost: 0.000670
Epoch  800/1000 W: 0.976, b: 0.054 Cost: 0.000414
Epoch  900/1000 W: 0.981, b: 0.042 Cost: 0.000256
Epoch 1000/1000 W: 0.985, b: 0.033 Cost: 0.000158


---

## High-level Implementation with `nn.Module`

Data 만들기

In [19]:
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

Linear regression 모델을 Pytorch의 nn.Module을 상속해서 만들어 보자.

In [20]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()      # 상속할 때 쓰는 구문
        self.linear = nn.Linear(1,1)
        
    def forward(self,x):
        return self.linear(x)

모델 정의 후 Hypothesis 및 Cost 구하기

In [21]:
model = LinearRegressionModel()

## Hypothesis

이제 모델을 생성해서 예측값  𝐻(𝑥) 를 구해보자

In [22]:
hypothesis = model(x_train)
print(hypothesis)

tensor([[0.0739],
        [0.5891],
        [1.1044]], grad_fn=<AddmmBackward>)


## Cost
이제 mean squared error (MSE) 로 cost를 구해보자. MSE 역시 PyTorch에서 기본적으로 제공한다.

In [23]:
cost = F.mse_loss(hypothesis, y_train)
print(cost)

tensor(2.1471, grad_fn=<MseLossBackward>)


## Gradient Descent

마지막 주어진 cost를 이용해 $H(x)$ 의 $W, b$ 를 바꾸어서 cost를 줄여봅니다.   
이때 PyTorch의 `torch.optim` 에 있는 `optimizer` 들 중 하나를 사용할 수 있습니다.

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

**항상 붙어다니는 3줄!!**

In [ ]:
## Training with Full Codeoptimizer.zero_grad()
cost.backward()
optimizer.step()

## Training with Full Code (모델을 이용)

In [25]:
# 데이터
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])
# 모델 초기화
model = LinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0:
        params = list(model.parameters())
        W = params[0].item()
        b = params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: -0.101, b: 0.508 Cost: 4.630286
Epoch  100/1000 W: 0.713, b: 0.653 Cost: 0.061555
Epoch  200/1000 W: 0.774, b: 0.514 Cost: 0.038037
Epoch  300/1000 W: 0.822, b: 0.404 Cost: 0.023505
Epoch  400/1000 W: 0.860, b: 0.317 Cost: 0.014525
Epoch  500/1000 W: 0.890, b: 0.250 Cost: 0.008975
Epoch  600/1000 W: 0.914, b: 0.196 Cost: 0.005546
Epoch  700/1000 W: 0.932, b: 0.154 Cost: 0.003427
Epoch  800/1000 W: 0.947, b: 0.121 Cost: 0.002118
Epoch  900/1000 W: 0.958, b: 0.095 Cost: 0.001309
Epoch 1000/1000 W: 0.967, b: 0.075 Cost: 0.000809
